In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

2025-08-13 02:36:45.008060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755052605.359387      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755052605.455949      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# --- 1. Muat data ---
print("--- Memuat Data ---")
df_train = pd.read_csv('/kaggle/input/bdc-dataset/df_train (1).csv')
df_test = pd.read_csv('/kaggle/input/bdc-dataset/df_test (1).csv')

--- Memuat Data ---


In [3]:
df_train

,prompt,essay,task_achievement,coherence_and_cohesion,lexical_resource,grammatical_range
0,Young people who commit crimes should be treat...,Deciding to choose among the potential ways of...,7.0,7.0,6.5,6.5
1,Young people who commit crimes should be treat...,"In this modern era, youngsters who commit offe...",7.0,7.0,7.0,7.0
2,Young people who commit crimes should be treat...,"In this modern era, youngsters who commit offe...",6.0,5.5,5.5,5.0
3,Young people who commit crimes should be treat...,It is suggested that punishments for immature ...,7.5,8.0,7.0,7.5
4,Young people who commit crimes should be treat...,There is a controversial dispute about whether...,6.5,6.0,6.0,5.5
...,...,...,...,...,...,...
9907,The best way to teach children to cooperate is...,It is proved that sports activities at school ...,7.0,7.0,6.5,6.5
9908,The best way to teach children to cooperate is...,Most people suggest that the prime way to teac...,7.0,7.0,6.5,6.5
9909,The best way to teach children to cooperate is...,"Few argue that, the best way is teachers train...",4.0,4.0,4.0,4.0
9910,"In today's world, people spend a lot of money ...",Nowadays people tend to invest huge amounts of...,6.5,7.0,6.5,6.0


In [4]:
# --- 2. Imputasi Missing Value ---
print("\n--- Imputasi Missing Value pada Kolom Target ---")
target_columns = ['task_achievement', 'coherence_and_cohesion', 'lexical_resource', 'grammatical_range']
imputer = SimpleImputer(strategy='mean')
df_train[target_columns] = imputer.fit_transform(df_train[target_columns])


--- Imputasi Missing Value pada Kolom Target ---


In [5]:
# --- 3. Tokenisasi dengan BERT ---
print("\n--- Tokenisasi dengan BERT ---")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
MAX_LENGTH = 256  # Panjang maksimal token, sesuaikan jika esai Anda sangat panjang

# Tokenisasi data training
tokenized_train = tokenizer(
    df_train['essay'].tolist(),
    padding=True,
    truncation=True,
    max_length=MAX_LENGTH,
    return_tensors='pt'
)

# Tokenisasi data test
tokenized_test = tokenizer(
    df_test['essay'].tolist(),
    padding=True,
    truncation=True,
    max_length=MAX_LENGTH,
    return_tensors='pt'
)


--- Tokenisasi dengan BERT ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
# --- 4. Persiapan Data untuk PyTorch ---
print("\n--- Persiapan Data untuk PyTorch ---")
train_labels = torch.tensor(df_train[target_columns].values, dtype=torch.float32)
train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], train_labels)

test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'])

BATCH_SIZE = 16  # Ukuran batch, sesuaikan dengan kapasitas GPU
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)


--- Persiapan Data untuk PyTorch ---


In [7]:
# --- 5. Definisi Model untuk Multi-Target Regresi ---
print("\n--- Definisi Model RoBERTa untuk Multi-Target Regresi ---")
from transformers import RobertaModel

class RobertaForMultiTargetRegression(torch.nn.Module):
    def __init__(self, num_labels):
        super(RobertaForMultiTargetRegression, self).__init__()
        # Menggunakan RobertaModel sebagai base feature extractor
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = torch.nn.Dropout(0.1)
        # Menambahkan lapisan regresi baru di atas output RoBERTa
        self.regressor = torch.nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        # Mengambil hidden state dari [CLS] token, yang merepresentasikan seluruh kalimat
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.regressor(cls_output)
        return logits

# Inisialisasi model, optimizer, dan loss function
num_labels = len(target_columns)
model = RobertaForMultiTargetRegression(num_labels)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
loss_fn = torch.nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


--- Definisi Model RoBERTa untuk Multi-Target Regresi ---


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForMultiTargetRegression(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# --- 6. Pelatihan Model (Fine-Tuning) ---
print("\n--- Memulai Pelatihan Model (Fine-Tuning) ---")
EPOCHS = 15 # Jumlah epoch, 3-5 adalah nilai yang umum
model.train()

for epoch in range(EPOCHS):
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        input_ids, attention_mask, labels = batch 
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
         
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_dataloader)
    print(f"Loss di akhir Epoch {epoch+1}: {avg_loss:.4f}")


--- Memulai Pelatihan Model (Fine-Tuning) ---


Epoch 1/8:   0%|          | 0/620 [00:00<?, ?it/s]

In [ ]:
print("\n--- Memulai Prediksi pada Data Test ---")
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Predicting"):
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        outputs = model(input_ids, attention_mask)
        test_predictions.extend(outputs.cpu().numpy())

In [ ]:
# --- 8. Simpan Hasil Prediksi ke File Excel ---
print("\n--- Menyimpan Hasil Prediksi ke csv ---")
predicted_df = pd.DataFrame(test_predictions, columns=[f'predicted_{col}' for col in target_columns])
final_predictions = pd.concat([df_test.reset_index(drop=True), predicted_df], axis=1)
final_predictions.to_csv('hasil_prediksi_esay_bert.csv', index=False)

print("\nProses selesai. File 'hasil_prediksi_esay_roberta.csv' berhasil dibuat.")